# Selección modelo de Machine Learning

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import make_scorer, mean_squared_error, r2_score

In [20]:
df = pd.read_csv("videogame_data_model.csv")
df.head()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,metacritic,Global_Sales,Franchise_AceCombat,Franchise_AdventureTime,Franchise_AgeOfEmpires,Franchise_AloneInTheDark,...,Publisher_Takara,Publisher_Take-Two Interactive,Publisher_Titus,Publisher_Wanadoo,Publisher_Xseed Games,Publisher_Zoo Games,Publisher_Zushi Games,esrb_rating_Mature,esrb_rating_Rating Pending,PC1
0,0.567773,0.203791,0.200948,0.027488,80.238683,10.55,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.171564
1,0.372330,0.312309,0.251272,0.064090,78.000000,9.83,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.341259
2,0.468817,0.183871,0.322581,0.024731,80.238683,9.30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.938007
3,0.722520,0.158177,0.093834,0.025469,80.238683,7.46,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.667963
4,0.493171,0.279211,0.156297,0.071320,85.000000,6.59,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.028092


## Modelos de Machine Learning

En el presente proyecto, se desea predecir la variable continua **Global_Sales**. Por lo tanto, el problema corresponde a un problema de regresión. De esta forma, las alternativas de modelos de regresión que se utilizarán son:

- Linear Regression
- Ridge Regression
- Gradient Boost (XGBoost)
- Gradient Boost (LightGBM)

En todas las evaluaciones de modelos se utilizará cross validation con **k=5** con las variables *X* e *y* que se muestran a continuación.

In [21]:
X = df.drop(columns=["Global_Sales"])
y = df["Global_Sales"]

#### Linear Regression

Se realiza una **cross-validation** con el modelo de `LinearRegression` utilizando `k=5`. Al observar los resultados de `cv_scores` se puede concluir que:

- El modelo de `LinearRegression` muestra un bajo coeficiente de correlacion $R^2$ con un promedio de 0.115, lo que significa que el modelo explica
aproximadamente un 11.5% de la variabilidad de los datos.

- En general el modelo es consistente, con una desviación estandar del coeficiende de correlacion $R^2$ de 0.06.

In [22]:
model = LinearRegression()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring="r2")

In [23]:
print("Puntajes r2 del Cross-Validation:", cv_scores)
print("Promedio r2:", np.mean(cv_scores))
print("Desviación Estándar de r2:", np.std(cv_scores))

Puntajes r2 del Cross-Validation: [-0.00089911  0.16465099  0.11044923  0.155781    0.14770199]
Promedio r2: 0.11553681806158707
Desviación Estándar de r2: 0.06107307839337174


#### Ridge Regression

Se realiza una **cross-validation** con el modelo de `Ridge` utilizando `k=5` con hiperparámetro `alpha=2.0` el cual fue seleccionado al tanteo evaluando el mayor promedio de $R^2$. Al observar los resultados de `cv_scores` se puede concluir que:

- El modelo de `Ridge` muestra un bajo coeficiente de correlacion $R^2$ con un promedio de 0.1617, lo que significa que el modelo explica
aproximadamente un 16.17% de la variabilidad de los datos. Esto es una mejora respecto del modelo `LinearRegression`

- En general el modelo es consistente, con una desviación estandar del coeficiende de correlacion $R^2$ de 0.01 siendo menor que en el modelo
`LinearRegression`

In [50]:
alpha = 2.0
model = Ridge(alpha=alpha)
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring="r2")

In [51]:
print("Puntajes r2 del Cross-Validation:", cv_scores)
print("Promedio r2:", np.mean(cv_scores))
print("Desviación Estándar de r2:", np.std(cv_scores))

Puntajes r2 del Cross-Validation: [0.16568205 0.18476329 0.15460895 0.15711501 0.14651893]
Promedio r2: 0.16173764579446698
Desviación Estándar de r2: 0.013034836057014724


#### Gradient Boost (XGBoost)